In [1]:
# Import libraries here
import sqlite3
import warnings
import zipfile
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from category_encoders import OneHotEncoder
from category_encoders import OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.utils.validation import check_is_fitted

In [29]:

from sqlite3 import Error


def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()


if __name__ == '__main__':
    create_connection("covid19.db")

2.6.0


## Connect to database

In [30]:
import sqlite3 as db
conn = db.connect('covid19.db')
%load_ext sql
%sql sqlite:///covid19.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [31]:
c = conn.cursor()

## Import data into pandas

In [87]:
df1=pd.read_csv("/home/tatenda/Desktop/dataset/covid-deaths.csv")
df2=pd.read_csv("/home/tatenda/Desktop/dataset/covid_vaccinations_1.csv")

In [15]:
df1.head()

,iso_code,continent,location,date,total_cases,population,new_cases,new_cases_smoothed,total_deaths,new_deaths,...,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million
0,BGR,Europe,Bulgaria,2021-08-01,425148.0,6885868.0,94.0,208.857,18215.0,2.0,...,0.436,1.69,78.0,11.328,805.0,116.906,NaN,NaN,NaN,NaN
1,BGR,Europe,Bulgaria,2021-08-02,425541.0,6885868.0,393.0,237.429,18222.0,7.0,...,0.477,1.70,81.0,11.763,810.0,117.632,NaN,NaN,NaN,NaN
2,BGR,Europe,Bulgaria,2021-08-03,426003.0,6885868.0,462.0,274.857,18225.0,3.0,...,0.456,1.71,82.0,11.908,851.0,123.586,NaN,NaN,NaN,NaN
3,BGR,Europe,Bulgaria,2021-08-04,426432.0,6885868.0,429.0,305.286,18229.0,4.0,...,0.498,1.71,86.0,12.489,914.0,132.736,NaN,NaN,NaN,NaN
4,BGR,Europe,Bulgaria,2021-08-05,426932.0,6885868.0,500.0,343.714,18243.0,14.0,...,0.726,1.70,90.0,13.070,955.0,138.690,NaN,NaN,NaN,NaN


In [88]:
df2.head()

,iso_code,continent,location,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,new_vaccinations_smoothed_per_million,new_people_vaccinated_smoothed,new_people_vaccinated_smoothed_per_hundred
0,BGR,Europe,Bulgaria,2021-08-01,2030301.0,1029158.0,1001143.0,NaN,2165.0,8363.0,29.49,14.95,14.54,NaN,1215.0,2561.0,0.037
1,BGR,Europe,Bulgaria,2021-08-02,2038611.0,1032438.0,1006173.0,NaN,8310.0,8332.0,29.61,14.99,14.61,NaN,1210.0,3000.0,0.044
2,BGR,Europe,Bulgaria,2021-08-03,2050991.0,1037737.0,1013254.0,NaN,12380.0,8884.0,29.79,15.07,14.71,NaN,1290.0,3728.0,0.054
3,BGR,Europe,Bulgaria,2021-08-04,2063643.0,1043316.0,1020327.0,NaN,12652.0,8838.0,29.97,15.15,14.82,NaN,1283.0,3730.0,0.054
4,BGR,Europe,Bulgaria,2021-08-05,2076157.0,1048780.0,1027377.0,NaN,12514.0,8881.0,30.15,15.23,14.92,NaN,1290.0,3761.0,0.055


<h3>Saving data frames  to sql tables</h3>

In [96]:
df1.to_sql("covid_deaths", conn, if_exists="replace")
df2.to_sql("covid_vaccinations", conn, if_exists="replace")

In [95]:
%%sql
PRAGMA table_info(covid_deaths);

 * sqlite:///covid19.db
   sqlite:///nepal.db
Done.


cid,name,type,notnull,dflt_value,pk
0,index,INTEGER,0,None,0
1,iso_code,TEXT,0,None,0
2,continent,TEXT,0,None,0
3,location,TEXT,0,None,0
4,date,TEXT,0,None,0
5,total_cases,REAL,0,None,0
6,population,REAL,0,None,0
7,new_cases,REAL,0,None,0
8,new_cases_smoothed,REAL,0,None,0
9,total_deaths,REAL,0,None,0


In [97]:
%%sql
PRAGMA table_info(covid_vaccinations);

 * sqlite:///covid19.db
   sqlite:///nepal.db
Done.


cid,name,type,notnull,dflt_value,pk
0,index,INTEGER,0,None,0
1,iso_code,TEXT,0,None,0
2,continent,TEXT,0,None,0
3,location,TEXT,0,None,0
4,date,TEXT,0,None,0
5,total_vaccinations,REAL,0,None,0
6,people_vaccinated,REAL,0,None,0
7,people_fully_vaccinated,REAL,0,None,0
8,total_boosters,REAL,0,None,0
9,new_vaccinations,REAL,0,None,0


Let's select the data we are going to use

In [33]:
%%sql
SELECT Location, date, total_cases,new_cases, total_deaths, population
FROM covid_deaths
order by 1,2
LIMIT 5

 * sqlite:///covid19.db
   sqlite:///nepal.db
Done.


location,date,total_cases,new_cases,total_deaths,population
Bulgaria,2021-08-01,425148.0,94.0,18215.0,6885868.0
Bulgaria,2021-08-02,425541.0,393.0,18222.0,6885868.0
Bulgaria,2021-08-03,426003.0,462.0,18225.0,6885868.0
Bulgaria,2021-08-04,426432.0,429.0,18229.0,6885868.0
Bulgaria,2021-08-05,426932.0,500.0,18243.0,6885868.0


Total Cases vs Total Deaths.
What is the probability that one will die if they contract covid-19 in their country?

In [52]:
%%sql
SELECT Location, date, total_cases, total_deaths, (total_deaths/total_cases)*100 as Death_Percentage
FROM covid_deaths
WHERE location = 'Zimbabwe'
order by 1,2
LIMIT 5

 * sqlite:///covid19.db
   sqlite:///nepal.db
Done.


location,date,total_cases,total_deaths,Death_Percentage
Zimbabwe,2020-03-20,1.0,None,None
Zimbabwe,2020-03-21,3.0,None,None
Zimbabwe,2020-03-22,3.0,None,None
Zimbabwe,2020-03-23,3.0,1.0,33.33333333333333
Zimbabwe,2020-03-24,3.0,1.0,33.33333333333333


What is the maximum Death_Percentage in Zimbabwe? 

In [65]:
%%sql
SELECT Location, date, total_cases, total_deaths, max(total_deaths/total_cases)*100 as Death_Percentage
FROM covid_deaths
WHERE location = 'Zimbabwe'
order by 1,2


 * sqlite:///covid19.db
   sqlite:///nepal.db
Done.


location,date,total_cases,total_deaths,Death_Percentage
Zimbabwe,2020-03-23,3.0,1.0,33.33333333333333


Let's look at infection Rate for the countries in our data.The list below contains top ten countries with the highest infection rates!


In [60]:
%%sql
SELECT Location, population, max(total_cases) as HighestInfectionCount, max((total_cases/population))*100 as Percentage_Infected
FROM covid_deaths
group by location, population
order by Percentage_Infected desc 
LIMIT 10
 

 * sqlite:///covid19.db
   sqlite:///nepal.db
Done.


location,population,HighestInfectionCount,Percentage_Infected
Faeroe Islands,52888.0,34658.0,65.53093329299652
Cyprus,896007.0,562911.0,62.82439757725107
Gibraltar,32670.0,19999.0,61.215182124273035
San Marino,33746.0,19983.0,59.21590707046761
Denmark,5854240.0,3239092.0,55.328992320096205
Iceland,370335.0,202502.0,54.68076201277222
Saint Pierre and Miquelon,5883.0,3112.0,52.898181200067995
Portugal,10290103.0,5356285.0,52.052783145124984
Slovenia,2119410.0,1089921.0,51.42567978824295
France,67422000.0,34055710.0,50.51127228501082


Let's look at infection rates in each continent.

In [76]:
%%sql
SELECT continent, population, max(total_cases) as HighestInfectionCount , max((total_cases/population))*100 as Percentage_Infected
FROM covid_deaths
group by continent
order by Percentage_Infected desc 


 * sqlite:///covid19.db
   sqlite:///nepal.db
Done.


continent,population,HighestInfectionCount,Percentage_Infected
Europe,52888.0,34055710.0,65.53093329299652
North America,5883.0,91961519.0,52.898181200067995
Asia,9291000.0,44107588.0,49.4634377354429
South America,3764.0,6278998.0,49.07013815090329
Africa,106470.0,4004555.0,42.772612003381234
Oceania,17003.0,1647826.0,36.00541080985708
None,447593544.0,582007961.0,35.88680336282956


Let's look at the top 10 countries with highest total Death_Count 

In [78]:
%%sql
SELECT location, continent,  max(cast(total_deaths as int)) as Total_Death_Count
FROM covid_deaths
WHERE continent is not null
group by location
order by Total_Death_Count desc 
LIMIT 10

 * sqlite:///covid19.db
   sqlite:///nepal.db
Done.


location,continent,Total_Death_Count
United States,North America,1032820
India,Asia,526600
Russia,Europe,374849
Mexico,North America,328128
Peru,South America,214431
United Kingdom,Europe,185245
Italy,Europe,172729
Indonesia,Asia,157060
France,Europe,152526
Germany,Europe,144717


Let's look at Total_Death_Count in each continent

In [79]:

%%sql
SELECT continent,  max(cast(total_deaths as int)) as Total_Death_Count
FROM covid_deaths
WHERE continent is not null
group by continent
order by Total_Death_Count desc 


 * sqlite:///covid19.db
   sqlite:///nepal.db
Done.


continent,Total_Death_Count
North America,1032820
Asia,526600
Europe,374849
South America,214431
Africa,101982
Oceania,1624


Let's look at global statistics. Let's first look at Death_Percentage per any given day.

In [83]:
%%sql
SELECT date, sum(new_cases) as total_cases, sum(cast(new_deaths as int)) as total_deaths, 
sum(cast(new_deaths as int))/sum(new_cases)*100 as Death_percentage
FROM covid_deaths
WHERE continent is not null
group by date
order by 1,2
LIMIT 70

 * sqlite:///covid19.db
   sqlite:///nepal.db
Done.


date,total_cases,total_deaths,Death_percentage
2020-01-01,None,None,None
2020-01-02,None,None,None
2020-01-03,None,None,None
2020-01-04,None,None,None
2020-01-05,None,None,None
2020-01-06,None,None,None
2020-01-07,None,None,None
2020-01-08,None,None,None
2020-01-09,None,None,None
2020-01-10,None,None,None


Let's join our tables

In [98]:
%%sql
SELECT *
FROM covid_deaths as d
JOIN covid_vaccinations as v ON d.location = v.location
and d.date=v.date



LIMIT 5

 * sqlite:///covid19.db
   sqlite:///nepal.db
Done.


index,iso_code,continent,location,date,total_cases,population,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,index_1,iso_code_1,continent_1,location_1,date_1,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,new_vaccinations_smoothed_per_million,new_people_vaccinated_smoothed,new_people_vaccinated_smoothed_per_hundred
0,BGR,Europe,Bulgaria,2021-08-01,425148.0,6885868.0,94.0,208.857,18215.0,2.0,3.0,61742.107,13.651,30.331,2645.2729999999997,0.29,0.436,1.69,78.0,11.328,805.0,116.906,None,None,None,None,0,BGR,Europe,Bulgaria,2021-08-01,2030301.0,1029158.0,1001143.0,None,2165.0,8363.0,29.49,14.95,14.54,None,1215.0,2561.0,0.037000000000000005
1,BGR,Europe,Bulgaria,2021-08-02,425541.0,6885868.0,393.0,237.429,18222.0,7.0,3.286,61799.181,57.073,34.481,2646.289,1.0170000000000001,0.47700000000000004,1.7,81.0,11.763,810.0,117.632,None,None,None,None,1,BGR,Europe,Bulgaria,2021-08-02,2038611.0,1032438.0,1006173.0,None,8310.0,8332.0,29.61,14.99,14.61,None,1210.0,3000.0,0.044000000000000004
2,BGR,Europe,Bulgaria,2021-08-03,426003.0,6885868.0,462.0,274.857,18225.0,3.0,3.1430000000000002,61866.275,67.094,39.916,2646.725,0.436,0.456,1.71,82.0,11.908,851.0,123.586,None,None,None,None,2,BGR,Europe,Bulgaria,2021-08-03,2050991.0,1037737.0,1013254.0,None,12380.0,8884.0,29.79,15.07,14.71,None,1290.0,3728.0,0.054000000000000006
3,BGR,Europe,Bulgaria,2021-08-04,426432.0,6885868.0,429.0,305.286,18229.0,4.0,3.429,61928.575999999994,62.302,44.335,2647.3059999999996,0.581,0.498,1.71,86.0,12.489,914.0,132.736,None,None,None,None,3,BGR,Europe,Bulgaria,2021-08-04,2063643.0,1043316.0,1020327.0,None,12652.0,8838.0,29.97,15.15,14.82,None,1283.0,3730.0,0.054000000000000006
4,BGR,Europe,Bulgaria,2021-08-05,426932.0,6885868.0,500.0,343.714,18243.0,14.0,5.0,62001.189000000006,72.612,49.916000000000004,2649.3390000000004,2.033,0.726,1.7,90.0,13.07,955.0,138.69,None,None,None,None,4,BGR,Europe,Bulgaria,2021-08-05,2076157.0,1048780.0,1027377.0,None,12514.0,8881.0,30.15,15.23,14.92,None,1290.0,3761.0,0.055


Let's now look at the total population vs vaccinations. We will use a rolling count for the vaccinations.

In [120]:
%%sql
SELECT d.continent, d.location, d.date, d.population, v.new_vaccinations, sum(cast( v.new_vaccinations as int))
OVER (Partition by d.location order by d.location, d.date) as RollingPeopleVaccinated
FROM covid_deaths as d
JOIN covid_vaccinations as v 
ON d.location = v.location
and d.date=v.date
WHERE d.continent is not null
order by 2,3

LIMIT 5

 * sqlite:///covid19.db
   sqlite:///nepal.db
Done.


continent,location,date,population,new_vaccinations,RollingPeopleVaccinated
Europe,Bulgaria,2021-08-01,6885868.0,2165.0,2165
Europe,Bulgaria,2021-08-02,6885868.0,8310.0,10475
Europe,Bulgaria,2021-08-03,6885868.0,12380.0,22855
Europe,Bulgaria,2021-08-04,6885868.0,12652.0,35507
Europe,Bulgaria,2021-08-05,6885868.0,12514.0,48021


Let's create views for Data Visualisations

In [121]:
%%sql
Create View PercentagePopulationVaccinated as
SELECT d.continent, d.location, d.date, d.population, v.new_vaccinations, sum(cast( v.new_vaccinations as int))
OVER (Partition by d.location order by d.location, d.date) as RollingPeopleVaccinated
FROM covid_deaths as d
JOIN covid_vaccinations as v 
ON d.location = v.location
and d.date=v.date
WHERE d.continent is not null
order by 2,3

 * sqlite:///covid19.db
   sqlite:///nepal.db
Done.


[]